Clase 8
======

Anuncios
--------------

Por favor llenen la siguiente [encuesta](https://docs.google.com/forms/d/e/1FAIpQLSd0DyU1DYb2y2KBshDuzOxrAf-f459NRsu9_lIwgyo83FmD5w/viewform?usp=sf_link).


Introducción
------------------


En la clase pasado vimos como rutas alternativas y ciclos resultan en más de una configuración de flujos siendo compatible con los flujos de entrada y salida de un red metabólica:

<img src="https://raw.githubusercontent.com/mrivas/ModelamientoDeProcesosBiotecnologicos/master/clase8_alternativasCiclos.png" width="400">

Hoy veremos dos herramientas para abordar esta ambiguedad en la solución de FBA:

* Flux variability análisis
* Flux sampling

Flux variability análisis
--------

FVA (flux variability análisis) encuentra el rango de cada flujo metabólico en el valor óptimo.

<img src="https://raw.githubusercontent.com/modcommet/Clases/master/metanogenesis.png" width="400">

In [ ]:
from cobra import Model, Reaction, Metabolite
cobra_model = Model('primordialMetabolism')
# Ractions
E1 = Reaction('E1')
E1.name = "Exchange reaction of CO2"
E1.subsystem = 'Entrance'
E1.lower_bound = 0.  
E1.upper_bound = 10.  
E2 = Reaction('E2')
E2.name = 'Exchange reaction of H2'
E2.subsystem = 'Entrance'
E2.lower_bound = 0.  
E2.upper_bound = 1000.  
R1 = Reaction('R1')
R1.name = 'First reaction'
R1.subsystem = 'Internal reaction'
R1.lower_bound = 0.  
R1.upper_bound = 1000. 
R2 = Reaction('R2')
R2.name = 'Second reaction'
R2.subsystem = 'Internal reaction'
R2.lower_bound = 0.  
R2.upper_bound = 1000.  
R3 = Reaction('R3')
R3.name = 'Third reaction'
R3.subsystem = 'Internal reaction'
R3.lower_bound = 0.  
R3.upper_bound = 1000. 
E3 = Reaction('E3')
E3.name = 'Exchange reaction of CH4'
E3.subsystem = 'Exit'
E3.lower_bound = 0.  
E3.upper_bound = 1000.  
E4 = Reaction('E4')
E4.name = 'Exchange reaction of F420'
E4.subsystem = 'Exit'
E4.lower_bound = 0. 
E4.upper_bound = 1000. 
E5 = Reaction('E5')
E5.name = 'Exchange reaction of H20'
E5.subsystem = 'Exit'
E5.lower_bound = 0.  
E5.upper_bound = 1000. 
# Metabolites
CO2 = Metabolite('CO2', formula='CO2',name='carbon dioxide',compartment='c')
H2 = Metabolite('H2', formula='H2',name='H2',compartment='c')
F420 = Metabolite('F420', formula='F420',name='F420',compartment='c')
I1 = Metabolite('I1', formula='I1',name='I1',compartment='c')
H2O = Metabolite('H2O', formula='H2O',name='H2O',compartment='c')
CH4 = Metabolite('CH4', formula='CH4',name='CH4',compartment='c')
# Adding metabolites to reactions
E1.add_metabolites({CO2: 1.0})
E2.add_metabolites({H2: 1.0})
E3.add_metabolites({CH4: -1.0})
E4.add_metabolites({F420: -2})
E5.add_metabolites({H2O: -1})
R1.add_metabolites({CO2:-1,H2: -1.0,F420:-1,I1:1})
R2.add_metabolites({H2:-1,F420:1,I1:-1,CH4:1})
R3.add_metabolites({F420:1,I1:1,CH4:-1,H2O:1})
# Adding reactions to model 
cobra_model.add_reaction(E1)
cobra_model.add_reaction(E2)
cobra_model.add_reaction(R1)
cobra_model.add_reaction(R2)
cobra_model.add_reaction(R3)
cobra_model.add_reaction(E3)
cobra_model.add_reaction(E4)
cobra_model.add_reaction(E5)
# Define objective function
cobra_model.objective = 'E3'

cobra_model.objective.direction = "max"

# Agregar una restriccion en donde E3 y E1 tengan valores fijos (10)

solution = cobra_model.optimize()

In [ ]:
from cobra.flux_analysis import flux_variability_analysis
#flux_variability_analysis(cobra_model, cobra_model.reactions[:8])

Flux sampling
-------

Otra alternativa para explorar el espacio de soluciones alternativas de FBA es tomar muestrasa aleatorias. Esto se llama Flux sampling.

In [ ]:
#from cobra.test import create_test_model
from cobra.sampling import sample
#s = sample(cobra_model, 100) # tomaremos 100 muestras aleatorias
#s.head(6)

In [ ]:
s.head(10)

In [ ]:
#Si quiero ver solo los valores de la reacción E1
s["R1"]

In [ ]:
#Si quiero ver el valor promedio de la reacción E1 a lo largo de las 100 muestras
s["E1"].hist() # hace el grafico del histograma.

Ejercicios
==========

1. Computa FBA de la red de *Escherichia coli* usando los valores por defecto de consumo de glucos y oxigeno.
2. Realiza FVA para la solución anterior.
3. Computa 100 muestras aleatorias del espacio de soluciones alternativas del punto 1, y luego calcula el valor promedio de todas sus reacciones.

Tarea optativa
--------------

4. Grafica los resultados usando ```escher```. En este caso, para cada flujo se debe calcular el valor promedio de las 100 muestras.

In [ ]:
# 1. Calcular los flujos de E coli usando FBA
import cobra.test
model = cobra.test.create_test_model("ecoli")

In [ ]:
solutionEcoli = model.optimize()

In [ ]:
model.summary()

In [ ]:
# 2. Calcular los rangos inferior y superior de los flujos de E. coli
FVA_model= flux_variability_analysis(model, model.reactions[:8]) 

In [ ]:
FVA_model

In [ ]:
reactionDeInteres = model.reactions.get_by_id("EX_glc__D_e")
FVA_model= flux_variability_analysis(model, reactionDeInteres)

In [ ]:
FVA_model

In [15]:
# 3. Flux sampling
from cobra.sampling import sample 
s=sample(model,10) 


In [20]:
# Para obtener el promedio de todas las reacciones
s.mean(axis=0)


DM_4crsol_c    1.127901e-06
DM_5drib_c     1.487452e-03
DM_aacald_c   -4.893362e-14
DM_amob_c      1.011571e-08
DM_mththf_c    9.092359e-03
                   ...     
ZN2abcpp       3.106326e-01
ZN2t3pp        2.664248e-01
ZN2tpp         5.076306e-01
ZNabcpp        6.942844e-02
Zn2tex         1.665472e-06
Length: 2583, dtype: float64

In [21]:
promedios=s.mean(axis=0)
promedios["DM_4crsol_c"]

1.1279013126271982e-06